In [1]:
import ctypes
import numpy as np
from scipy.sparse import csc_matrix, coo_matrix
# Load the CHOLMOD DLL (adjust path and name for your system)
DIR = r"C:\Users\wyattluke.lowery\OneDrive - Texas A&M University\Research\GSP\Sparse DLLs\SuiteSparse\SuiteSparse\bin"
libcholmod = ctypes.CDLL(f"{DIR}\cholmod.dll")  # or .so on Linux, .dylib on Mac


In [4]:
# Load the types
c_double = ctypes.c_double
c_int = ctypes.c_int
c_void_p = ctypes.c_void_p

CHOLMOD_REAL = 0
CHOLMOD_MAXMETHODS = 9
CHOLMOD_HOST_SUPERNODE_BUFFERS = 8  # typical default

# GPU placeholders (when CUDA is not in use)
CHOLMOD_CUBLAS_HANDLE = ctypes.c_void_p
CHOLMOD_CUDASTREAM = ctypes.c_void_p
CHOLMOD_CUDAEVENT = ctypes.c_void_p    

class cholmod_method_struct(ctypes.Structure):
    _fields_ = [
        ("lnz", ctypes.c_double),
        ("fl", ctypes.c_double),
        ("prune_dense", ctypes.c_double),
        ("prune_dense2", ctypes.c_double),
        ("nd_oksep", ctypes.c_double),
        ("other_1", ctypes.c_double * 4),
        ("nd_small", ctypes.c_size_t),
        ("other_2", ctypes.c_double * 4),
        ("aggressive", ctypes.c_int),
        ("order_for_lu", ctypes.c_int),
        ("nd_compress", ctypes.c_int),
        ("nd_camd", ctypes.c_int),
        ("nd_components", ctypes.c_int),
        ("ordering", ctypes.c_int),
        ("other_3", ctypes.c_size_t * 4),
    ]

class cholmod_common(ctypes.Structure):
    _fields_ = [
        # primary parameters
        ("dbound", ctypes.c_double),
        ("grow0", ctypes.c_double),
        ("grow1", ctypes.c_double),
        ("grow2", ctypes.c_size_t),
        ("maxrank", ctypes.c_size_t),
        ("supernodal_switch", ctypes.c_double),
        ("supernodal", ctypes.c_int),
        ("final_asis", ctypes.c_int),
        ("final_super", ctypes.c_int),
        ("final_ll", ctypes.c_int),
        ("final_pack", ctypes.c_int),
        ("final_monotonic", ctypes.c_int),
        ("final_resymbol", ctypes.c_int),
        ("zrelax", ctypes.c_double * 3),
        ("nrelax", ctypes.c_size_t * 3),
        ("prefer_zomplex", ctypes.c_int),
        ("prefer_upper", ctypes.c_int),
        ("quick_return_if_not_posdef", ctypes.c_int),
        ("prefer_binary", ctypes.c_int),

        # printing and error handling
        ("print", ctypes.c_int),
        ("precise", ctypes.c_int),
        ("try_catch", ctypes.c_int),
        ("error_handler", ctypes.c_void_p),  # pointer to error handler function

        # ordering options
        ("nmethods", ctypes.c_int),
        ("current", ctypes.c_int),
        ("selected", ctypes.c_int),
        ("method", cholmod_method_struct * (CHOLMOD_MAXMETHODS + 1)),
        ("postorder", ctypes.c_int),
        ("default_nesdis", ctypes.c_int),

        # METIS workarounds
        ("metis_memory", ctypes.c_double),
        ("metis_dswitch", ctypes.c_double),
        ("metis_nswitch", ctypes.c_size_t),

        # workspace
        ("nrow", ctypes.c_size_t),
        ("mark", ctypes.c_int64),
        ("iworksize", ctypes.c_size_t),
        ("xworkbytes", ctypes.c_size_t),
        ("Flag", ctypes.c_void_p),
        ("Head", ctypes.c_void_p),
        ("Xwork", ctypes.c_void_p),
        ("Iwork", ctypes.c_void_p),
        ("itype", ctypes.c_int),
        ("other_5", ctypes.c_int),
        ("no_workspace_reallocate", ctypes.c_int),

        # statistics
        ("status", ctypes.c_int),
        ("fl", ctypes.c_double),
        ("lnz", ctypes.c_double),
        ("anz", ctypes.c_double),
        ("modfl", ctypes.c_double),
        ("malloc_count", ctypes.c_size_t),
        ("memory_usage", ctypes.c_size_t),
        ("memory_inuse", ctypes.c_size_t),
        ("nrealloc_col", ctypes.c_double),
        ("nrealloc_factor", ctypes.c_double),
        ("ndbounds_hit", ctypes.c_double),
        ("rowfacfl", ctypes.c_double),
        ("aatfl", ctypes.c_double),
        ("called_nd", ctypes.c_int),
        ("blas_ok", ctypes.c_int),

        # SuiteSparseQR control/statistics
        ("SPQR_grain", ctypes.c_double),
        ("SPQR_small", ctypes.c_double),
        ("SPQR_shrink", ctypes.c_int),
        ("SPQR_nthreads", ctypes.c_int),
        ("SPQR_flopcount", ctypes.c_double),
        ("SPQR_analyze_time", ctypes.c_double),
        ("SPQR_factorize_time", ctypes.c_double),
        ("SPQR_solve_time", ctypes.c_double),
        ("SPQR_flopcount_bound", ctypes.c_double),
        ("SPQR_tol_used", ctypes.c_double),
        ("SPQR_norm_E_fro", ctypes.c_double),
        ("SPQR_istat", ctypes.c_int64 * 8),

        # CHOLMOD v5.0 additions
        ("nsbounds_hit", ctypes.c_double),
        ("sbound", ctypes.c_float),
        ("other_6", ctypes.c_float),

        # GPU configuration and statistics
        ("useGPU", ctypes.c_int),
        ("maxGpuMemBytes", ctypes.c_size_t),
        ("maxGpuMemFraction", ctypes.c_double),
        ("gpuMemorySize", ctypes.c_size_t),
        ("gpuKernelTime", ctypes.c_double),
        ("gpuFlops", ctypes.c_int64),
        ("gpuNumKernelLaunches", ctypes.c_int),
        ("cublasHandle", CHOLMOD_CUBLAS_HANDLE),
        ("gpuStream", CHOLMOD_CUDASTREAM * CHOLMOD_HOST_SUPERNODE_BUFFERS),
        ("cublasEventPotrf", CHOLMOD_CUDAEVENT * 3),
        ("updateCKernelsComplete", CHOLMOD_CUDAEVENT),
        ("updateCBuffersFree", CHOLMOD_CUDAEVENT * CHOLMOD_HOST_SUPERNODE_BUFFERS),
        ("dev_mempool", ctypes.c_void_p),
        ("dev_mempool_size", ctypes.c_size_t),
        ("host_pinned_mempool", ctypes.c_void_p),
        ("host_pinned_mempool_size", ctypes.c_size_t),
        ("devBuffSize", ctypes.c_size_t),
        ("ibuffer", ctypes.c_int),
        ("syrkStart", ctypes.c_double),
        ("cholmod_cpu_gemm_time", ctypes.c_double),
        ("cholmod_cpu_syrk_time", ctypes.c_double),
        ("cholmod_cpu_trsm_time", ctypes.c_double),
        ("cholmod_cpu_potrf_time", ctypes.c_double),
        ("cholmod_gpu_gemm_time", ctypes.c_double),
        ("cholmod_gpu_syrk_time", ctypes.c_double),
        ("cholmod_gpu_trsm_time", ctypes.c_double),
        ("cholmod_gpu_potrf_time", ctypes.c_double),
        ("cholmod_assemble_time", ctypes.c_double),
        ("cholmod_assemble_time2", ctypes.c_double),
        ("cholmod_cpu_gemm_calls", ctypes.c_size_t),
        ("cholmod_cpu_syrk_calls", ctypes.c_size_t),
        ("cholmod_cpu_trsm_calls", ctypes.c_size_t),
        ("cholmod_cpu_potrf_calls", ctypes.c_size_t),
        ("cholmod_gpu_gemm_calls", ctypes.c_size_t),
        ("cholmod_gpu_syrk_calls", ctypes.c_size_t),
        ("cholmod_gpu_trsm_calls", ctypes.c_size_t),
        ("cholmod_gpu_potrf_calls", ctypes.c_size_t),
        ("chunk", ctypes.c_double),
        ("nthreads_max", ctypes.c_int),
        # blas_dump omitted (only used if CHOLMOD compiled with -DBLAS_DUMP)
    ]

class cholmod_factor(ctypes.Structure):
    _fields_ = [
        # Factor size
        ("n", ctypes.c_size_t),
        ("minor", ctypes.c_size_t),

        # Symbolic ordering and analysis
        ("Perm", ctypes.c_void_p),       # int32/int64 array of size n
        ("ColCount", ctypes.c_void_p),   # int32/int64 array of size n
        ("IPerm", ctypes.c_void_p),      # int32/int64 array of size n

        # Simplicial factorization
        ("nzmax", ctypes.c_size_t),      # # entries L->i, L->x, L->z can hold
        ("p", ctypes.c_void_p),          # int32/int64, size n+1
        ("i", ctypes.c_void_p),          # int32/int64, size nzmax
        ("x", ctypes.c_void_p),          # float/double, size nzmax or 2*nzmax
        ("z", ctypes.c_void_p),          # float/double, size nzmax or empty
        ("nz", ctypes.c_void_p),         # int32/int64, size ncol
        ("next", ctypes.c_void_p),       # int32/int64, size n+2
        ("prev", ctypes.c_void_p),       # int32/int64, size n+2

        # Supernodal factorization
        ("nsuper", ctypes.c_size_t),     # # supernodes
        ("ssize", ctypes.c_size_t),      # # integers in L->s
        ("xsize", ctypes.c_size_t),      # # entries in L->x
        ("maxcsize", ctypes.c_size_t),   # size of largest update matrix
        ("maxesize", ctypes.c_size_t),   # max # rows in supernodes excl. triangular
        ("super", ctypes.c_void_p),      # int32/int64, size nsuper+1
        ("pi", ctypes.c_void_p),         # int32/int64, size nsuper+1
        ("px", ctypes.c_void_p),         # int32/int64, size nsuper+1
        ("s", ctypes.c_void_p),          # int32/int64, size ssize

        # Type of factorization
        ("ordering", ctypes.c_int),      # fill-reducing ordering method
        ("is_ll", ctypes.c_int),         # 1 if LL', 0 if LDL'
        ("is_super", ctypes.c_int),      # 1 if supernodal, 0 if simplicial
        ("is_monotonic", ctypes.c_int),  # 1 if columns appear 0..n-1
        ("itype", ctypes.c_int),         # int type for Perm, ColCount, etc.
        ("xtype", ctypes.c_int),         # pattern, real, complex, zomplex
        ("dtype", ctypes.c_int),         # double/single
        ("useGPU", ctypes.c_int),        # symbolic factorization may use GPU
    ]

class cholmod_sparse(ctypes.Structure):
    _fields_ = [
        ("nrow", ctypes.c_size_t),     # number of rows
        ("ncol", ctypes.c_size_t),     # number of columns
        ("nzmax", ctypes.c_size_t),    # maximum number of entries
        ("p", ctypes.c_void_p),        # column pointers
        ("i", ctypes.c_void_p),        # row indices
        ("nz", ctypes.c_void_p),        # numeric values
        ("x", ctypes.c_void_p),        # numeric values
        ("z", ctypes.c_void_p),        # complex values
        ("stype", ctypes.c_int),       # symmetric flag
        ("itype", ctypes.c_int),       # index type
        ("xtype", ctypes.c_int),       # real/pattern/complex
        ("dtype", ctypes.c_int),       # data type
        ("sorted", ctypes.c_int),      # columns sorted
        ("packed", ctypes.c_int)       # packed/unpacked
    ]

class cholmod_dense(ctypes.Structure):
    _fields_ = [
        ("nrow", ctypes.c_size_t),     # number of rows
        ("ncol", ctypes.c_size_t),     # number of columns
        ("nzmax", ctypes.c_size_t),    # maximum entries
        ("d", ctypes.c_size_t),        # leading dimension
        ("x", ctypes.c_void_p),        # values
        ("z", ctypes.c_void_p),        # complex values
        ("xtype", ctypes.c_int),       # real/pattern/complex
        ("dtype", ctypes.c_int)        # data type
    ]


CHOLMOD_A   = 0 #  0  /* solve Ax=b    */
#define CHOLMOD_LDLt 1  /* solve LDL'x=b */
#define CHOLMOD_LD   2  /* solve LDx=b   */
#define CHOLMOD_DLt  3  /* solve DL'x=b  */
#define CHOLMOD_L    4  /* solve Lx=b    */
#define CHOLMOD_Lt   5  /* solve L'x=b   */
#define CHOLMOD_D    6  /* solve Dx=b    */
#define CHOLMOD_P    7  /* permute x=Px  */
#define CHOLMOD_Pt   8  /* permute x=P'x */

class CholTool:

    def __init__(self, dll, A) -> None:
        ''' 
        dll: the ctype dll compiles CHOLMOD 
        A: csc_matrix - the working matrix, which can undergo lowrank changes
        '''


        self.dll = dll
        self.A = self.to_cholmod_sparse(A) # Parse to Cholmod format
        self.common = cholmod_common()

        # Syntax Sugar Pointer Names
        self.A_ptr = ctypes.byref(self.A)
        self.common_ptr = ctypes.byref(self.common)

        # DLL Setup    
        self.init_arg_types()
        self.init_return_types()

        # TODO improve
        self.MODE = CHOLMOD_A


    def status(self):
        ''' 
        Cholmod Status: 
        0 OK; 
        -4 Invalid Input; 
        -2 Out of Mem
        '''
        stat = self.common.status
        print("CHOLMOD status:", stat)
        return stat
    
    def to_cholmod_sparse(self, A, itype=0, dtype=0):
        """
        Convert a 2D NumPy array A into a cholmod_sparse struct.
        
        Parameters:
            A : np.ndarray
                Dense or 2D array to convert.
            itype : int
                0=int32 indices, 1=int64 indices
            dtype : int
                0=double (real), 1=float (single), 2=complex
        
        Returns:
            cholmod_sparse instance.
        """
        # Convert to CSC format
        nrow, ncol = A.shape
        
        # Prepare contiguous arrays
        x = np.ascontiguousarray(A.data, dtype=np.float64)
        i = np.ascontiguousarray(A.indices, dtype=np.int32 if itype==0 else np.int64)
        p = np.ascontiguousarray(A.indptr, dtype=np.int32 if itype==0 else np.int64)
        
        # Cast to void pointers for ctypes
        x_ptr = x.ctypes.data_as(ctypes.c_void_p)
        i_ptr = i.ctypes.data_as(ctypes.c_void_p)
        p_ptr = p.ctypes.data_as(ctypes.c_void_p)
        
        # Initialize struct
        cholA = cholmod_sparse()
        cholA.nrow = nrow
        cholA.ncol = ncol
        cholA.nzmax = len(x)
        cholA.p = p_ptr
        cholA.i = i_ptr
        cholA.x = x_ptr
        cholA.z = None             # None if real
        cholA.stype = 0            # 0 = general
        cholA.itype = itype
        cholA.xtype = 1            # 1 = real
        cholA.dtype = dtype
        cholA.sorted = 1           # columns sorted
        cholA.packed = 1           # packed
        
        return cholA
    
    def to_dense_vector(self, values):
        n = len(values)
        D = self.dll.cholmod_allocate_dense(
            ctypes.c_size_t(n), ctypes.c_size_t(1), ctypes.c_size_t(n),
            ctypes.c_int(1),  # xtype: real
            self.common_ptr
        )
        if not D:
            raise MemoryError("CHOLMOD allocate dense failed")
        arr = (ctypes.c_double * n)(*values)
        D.contents.x = ctypes.cast(arr, ctypes.c_void_p)
        return D
    
    def solve(self, b):

        x = self.dll.cholmod_solve(
            self.MODE, 
            self.fact, 
            self.to_dense_vector(b), 
            self.common_ptr
        )

        nsol = x.contents.nrow
        sol = np.ctypeslib.as_array(ctypes.cast(x.contents.x, ctypes.POINTER(ctypes.c_double)), shape=(nsol,))

        return sol

    def __enter__(self):

        # Start up CHOLMOD
        self.dll.cholmod_start(ctypes.byref(self.common))

        # Analyze
        factor = self.dll.cholmod_analyze(self.A_ptr, self.common_ptr)
        if not factor:
            raise RuntimeError("CHOLMOD analyze failed")
        
        self.fact     = factor 
        self.fact_ptr = ctypes.byref(self.fact)
        

        # Factorize
        if self.dll.cholmod_factorize(self.A_ptr, factor, self.common_ptr) == 0:
            raise RuntimeError("CHOLMOD factorize failed")


        return self
    
    def numeric_factorization(self, beta, fset=None, fsize=0):
        ''' 
        Assumes a symbolic factorization has already occured,
        which is computed by default when this class enters a context.
        '''

        # Must be complex for DLL use
        beta_cmplx = (ctypes.c_double * 2)(beta, 0.0) 

        self.dll.cholmod_factorize_p(
            self.A_ptr,
            beta_cmplx,
            fset,
            fsize,
            self.fact,
            self.common_ptr
        )
    
    def __exit__(self, type, value, traceback):

        # Free Factored Object
        self.dll.cholmod_free_factor(self.fact, self.common_ptr)

        # TODO FREE MATRIX AND DENSE
        #libcholmod.cholmod_free_sparse(ctypes.byref(A), ctypes.byref(common))
        #libcholmod.cholmod_free_dense(ctypes.byref(b), ctypes.byref(common))

        # Finish
        self.dll.cholmod_finish(ctypes.byref(self.common))

    def init_arg_types(self):
        self.dll.cholmod_start.argtypes = [ctypes.POINTER(cholmod_common)]
        self.dll.cholmod_finish.argtypes = [ctypes.POINTER(cholmod_common)]

        self.dll.cholmod_allocate_sparse.argtypes = [
            ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t,
            ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int,
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_analyze.argtypes = [
            ctypes.POINTER(cholmod_sparse),
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_factorize.argtypes = [
            ctypes.POINTER(cholmod_sparse),
            ctypes.POINTER(cholmod_factor),
            ctypes.POINTER(cholmod_common)
        ]
        # NOTE This is the shifting numeric factorization
        self.dll.cholmod_factorize_p.argtypes = [
            ctypes.POINTER(cholmod_sparse),          # A
            ctypes.POINTER(ctypes.c_double),         # beta[2]
            ctypes.POINTER(ctypes.c_int32),           # fset (int32_t*)
            ctypes.c_size_t,                          # fsize
            ctypes.POINTER(cholmod_factor),           # L
            ctypes.POINTER(cholmod_common)            # Common
        ]

        self.dll.cholmod_solve.argtypes = [
            ctypes.c_int,
            ctypes.POINTER(cholmod_factor),
            ctypes.POINTER(cholmod_dense),
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_allocate_dense.argtypes = [
            ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t, ctypes.c_int,
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_free_sparse.argtypes = [
            ctypes.POINTER(ctypes.POINTER(cholmod_sparse)),
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_free_dense.argtypes = [
            ctypes.POINTER(ctypes.POINTER(cholmod_dense)),
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_free_factor.argtypes = [
            ctypes.POINTER(ctypes.POINTER(cholmod_factor)),
            ctypes.POINTER(cholmod_common)
        ]
        self.dll.cholmod_norm_sparse.argtypes = [
            ctypes.POINTER(cholmod_sparse),  # A
            ctypes.c_int,                    # norm type: 0=inf, 1=1
            ctypes.POINTER(cholmod_common)   # Common
        ]

    def init_return_types(self):
        self.dll.cholmod_start.restype = None
        self.dll.cholmod_finish.restype = None
        self.dll.cholmod_allocate_sparse.restype = ctypes.POINTER(cholmod_sparse)
        self.dll.cholmod_analyze.restype = ctypes.POINTER(cholmod_factor)
        self.dll.cholmod_factorize.restype = ctypes.c_int
        self.dll.cholmod_factorize_p.restype = ctypes.c_int
        self.dll.cholmod_solve.restype = ctypes.POINTER(cholmod_dense)
        self.dll.cholmod_allocate_dense.restype = ctypes.POINTER(cholmod_dense)
        self.dll.cholmod_free_sparse.restype = None
        self.dll.cholmod_free_dense.restype = None
        self.dll.cholmod_free_factor.restype = None
        self.dll.cholmod_norm_sparse.restype = ctypes.c_double

In [5]:
A = csc_matrix(np.array([
    [1, 0, 2],
    [0, 3, 0],
    [4, 0, 5]
]))
b = [1.0,1.0,1.0]

with CholTool(libcholmod, A) as c: 

    x = c.solve(b)

    for val in x: print(f"{val:.3f}")

3.000
0.111
-1.000


Setup

In [ ]:

''' 
STRUCTS
'''

# Load the types
c_double = ctypes.c_double
c_int = ctypes.c_int
c_void_p = ctypes.c_void_p


# Basic scalar (double)
CHOLMOD_REAL = 0

CHOLMOD_MAXMETHODS = 9
CHOLMOD_HOST_SUPERNODE_BUFFERS = 8  # typical default

# GPU placeholders (when CUDA is not in use)
CHOLMOD_CUBLAS_HANDLE = ctypes.c_void_p
CHOLMOD_CUDASTREAM = ctypes.c_void_p
CHOLMOD_CUDAEVENT = ctypes.c_void_p

class cholmod_method_struct(ctypes.Structure):
    _fields_ = [
        ("lnz", ctypes.c_double),
        ("fl", ctypes.c_double),
        ("prune_dense", ctypes.c_double),
        ("prune_dense2", ctypes.c_double),
        ("nd_oksep", ctypes.c_double),
        ("other_1", ctypes.c_double * 4),
        ("nd_small", ctypes.c_size_t),
        ("other_2", ctypes.c_double * 4),
        ("aggressive", ctypes.c_int),
        ("order_for_lu", ctypes.c_int),
        ("nd_compress", ctypes.c_int),
        ("nd_camd", ctypes.c_int),
        ("nd_components", ctypes.c_int),
        ("ordering", ctypes.c_int),
        ("other_3", ctypes.c_size_t * 4),
    ]

class cholmod_common(ctypes.Structure):
    _fields_ = [
        # primary parameters
        ("dbound", ctypes.c_double),
        ("grow0", ctypes.c_double),
        ("grow1", ctypes.c_double),
        ("grow2", ctypes.c_size_t),
        ("maxrank", ctypes.c_size_t),
        ("supernodal_switch", ctypes.c_double),
        ("supernodal", ctypes.c_int),
        ("final_asis", ctypes.c_int),
        ("final_super", ctypes.c_int),
        ("final_ll", ctypes.c_int),
        ("final_pack", ctypes.c_int),
        ("final_monotonic", ctypes.c_int),
        ("final_resymbol", ctypes.c_int),
        ("zrelax", ctypes.c_double * 3),
        ("nrelax", ctypes.c_size_t * 3),
        ("prefer_zomplex", ctypes.c_int),
        ("prefer_upper", ctypes.c_int),
        ("quick_return_if_not_posdef", ctypes.c_int),
        ("prefer_binary", ctypes.c_int),

        # printing and error handling
        ("print", ctypes.c_int),
        ("precise", ctypes.c_int),
        ("try_catch", ctypes.c_int),
        ("error_handler", ctypes.c_void_p),  # pointer to error handler function

        # ordering options
        ("nmethods", ctypes.c_int),
        ("current", ctypes.c_int),
        ("selected", ctypes.c_int),
        ("method", cholmod_method_struct * (CHOLMOD_MAXMETHODS + 1)),
        ("postorder", ctypes.c_int),
        ("default_nesdis", ctypes.c_int),

        # METIS workarounds
        ("metis_memory", ctypes.c_double),
        ("metis_dswitch", ctypes.c_double),
        ("metis_nswitch", ctypes.c_size_t),

        # workspace
        ("nrow", ctypes.c_size_t),
        ("mark", ctypes.c_int64),
        ("iworksize", ctypes.c_size_t),
        ("xworkbytes", ctypes.c_size_t),
        ("Flag", ctypes.c_void_p),
        ("Head", ctypes.c_void_p),
        ("Xwork", ctypes.c_void_p),
        ("Iwork", ctypes.c_void_p),
        ("itype", ctypes.c_int),
        ("other_5", ctypes.c_int),
        ("no_workspace_reallocate", ctypes.c_int),

        # statistics
        ("status", ctypes.c_int),
        ("fl", ctypes.c_double),
        ("lnz", ctypes.c_double),
        ("anz", ctypes.c_double),
        ("modfl", ctypes.c_double),
        ("malloc_count", ctypes.c_size_t),
        ("memory_usage", ctypes.c_size_t),
        ("memory_inuse", ctypes.c_size_t),
        ("nrealloc_col", ctypes.c_double),
        ("nrealloc_factor", ctypes.c_double),
        ("ndbounds_hit", ctypes.c_double),
        ("rowfacfl", ctypes.c_double),
        ("aatfl", ctypes.c_double),
        ("called_nd", ctypes.c_int),
        ("blas_ok", ctypes.c_int),

        # SuiteSparseQR control/statistics
        ("SPQR_grain", ctypes.c_double),
        ("SPQR_small", ctypes.c_double),
        ("SPQR_shrink", ctypes.c_int),
        ("SPQR_nthreads", ctypes.c_int),
        ("SPQR_flopcount", ctypes.c_double),
        ("SPQR_analyze_time", ctypes.c_double),
        ("SPQR_factorize_time", ctypes.c_double),
        ("SPQR_solve_time", ctypes.c_double),
        ("SPQR_flopcount_bound", ctypes.c_double),
        ("SPQR_tol_used", ctypes.c_double),
        ("SPQR_norm_E_fro", ctypes.c_double),
        ("SPQR_istat", ctypes.c_int64 * 8),

        # CHOLMOD v5.0 additions
        ("nsbounds_hit", ctypes.c_double),
        ("sbound", ctypes.c_float),
        ("other_6", ctypes.c_float),

        # GPU configuration and statistics
        ("useGPU", ctypes.c_int),
        ("maxGpuMemBytes", ctypes.c_size_t),
        ("maxGpuMemFraction", ctypes.c_double),
        ("gpuMemorySize", ctypes.c_size_t),
        ("gpuKernelTime", ctypes.c_double),
        ("gpuFlops", ctypes.c_int64),
        ("gpuNumKernelLaunches", ctypes.c_int),
        ("cublasHandle", CHOLMOD_CUBLAS_HANDLE),
        ("gpuStream", CHOLMOD_CUDASTREAM * CHOLMOD_HOST_SUPERNODE_BUFFERS),
        ("cublasEventPotrf", CHOLMOD_CUDAEVENT * 3),
        ("updateCKernelsComplete", CHOLMOD_CUDAEVENT),
        ("updateCBuffersFree", CHOLMOD_CUDAEVENT * CHOLMOD_HOST_SUPERNODE_BUFFERS),
        ("dev_mempool", ctypes.c_void_p),
        ("dev_mempool_size", ctypes.c_size_t),
        ("host_pinned_mempool", ctypes.c_void_p),
        ("host_pinned_mempool_size", ctypes.c_size_t),
        ("devBuffSize", ctypes.c_size_t),
        ("ibuffer", ctypes.c_int),
        ("syrkStart", ctypes.c_double),
        ("cholmod_cpu_gemm_time", ctypes.c_double),
        ("cholmod_cpu_syrk_time", ctypes.c_double),
        ("cholmod_cpu_trsm_time", ctypes.c_double),
        ("cholmod_cpu_potrf_time", ctypes.c_double),
        ("cholmod_gpu_gemm_time", ctypes.c_double),
        ("cholmod_gpu_syrk_time", ctypes.c_double),
        ("cholmod_gpu_trsm_time", ctypes.c_double),
        ("cholmod_gpu_potrf_time", ctypes.c_double),
        ("cholmod_assemble_time", ctypes.c_double),
        ("cholmod_assemble_time2", ctypes.c_double),
        ("cholmod_cpu_gemm_calls", ctypes.c_size_t),
        ("cholmod_cpu_syrk_calls", ctypes.c_size_t),
        ("cholmod_cpu_trsm_calls", ctypes.c_size_t),
        ("cholmod_cpu_potrf_calls", ctypes.c_size_t),
        ("cholmod_gpu_gemm_calls", ctypes.c_size_t),
        ("cholmod_gpu_syrk_calls", ctypes.c_size_t),
        ("cholmod_gpu_trsm_calls", ctypes.c_size_t),
        ("cholmod_gpu_potrf_calls", ctypes.c_size_t),
        ("chunk", ctypes.c_double),
        ("nthreads_max", ctypes.c_int),
        # blas_dump omitted (only used if CHOLMOD compiled with -DBLAS_DUMP)
    ]

# CHOLMOD factor struct
class cholmod_factor(ctypes.Structure):
    _fields_ = [
        # Factor size
        ("n", ctypes.c_size_t),
        ("minor", ctypes.c_size_t),

        # Symbolic ordering and analysis
        ("Perm", ctypes.c_void_p),       # int32/int64 array of size n
        ("ColCount", ctypes.c_void_p),   # int32/int64 array of size n
        ("IPerm", ctypes.c_void_p),      # int32/int64 array of size n

        # Simplicial factorization
        ("nzmax", ctypes.c_size_t),      # # entries L->i, L->x, L->z can hold
        ("p", ctypes.c_void_p),          # int32/int64, size n+1
        ("i", ctypes.c_void_p),          # int32/int64, size nzmax
        ("x", ctypes.c_void_p),          # float/double, size nzmax or 2*nzmax
        ("z", ctypes.c_void_p),          # float/double, size nzmax or empty
        ("nz", ctypes.c_void_p),         # int32/int64, size ncol
        ("next", ctypes.c_void_p),       # int32/int64, size n+2
        ("prev", ctypes.c_void_p),       # int32/int64, size n+2

        # Supernodal factorization
        ("nsuper", ctypes.c_size_t),     # # supernodes
        ("ssize", ctypes.c_size_t),      # # integers in L->s
        ("xsize", ctypes.c_size_t),      # # entries in L->x
        ("maxcsize", ctypes.c_size_t),   # size of largest update matrix
        ("maxesize", ctypes.c_size_t),   # max # rows in supernodes excl. triangular
        ("super", ctypes.c_void_p),      # int32/int64, size nsuper+1
        ("pi", ctypes.c_void_p),         # int32/int64, size nsuper+1
        ("px", ctypes.c_void_p),         # int32/int64, size nsuper+1
        ("s", ctypes.c_void_p),          # int32/int64, size ssize

        # Type of factorization
        ("ordering", ctypes.c_int),      # fill-reducing ordering method
        ("is_ll", ctypes.c_int),         # 1 if LL', 0 if LDL'
        ("is_super", ctypes.c_int),      # 1 if supernodal, 0 if simplicial
        ("is_monotonic", ctypes.c_int),  # 1 if columns appear 0..n-1
        ("itype", ctypes.c_int),         # int type for Perm, ColCount, etc.
        ("xtype", ctypes.c_int),         # pattern, real, complex, zomplex
        ("dtype", ctypes.c_int),         # double/single
        ("useGPU", ctypes.c_int),        # symbolic factorization may use GPU
    ]


class cholmod_sparse(ctypes.Structure):
    _fields_ = [
        ("nrow", ctypes.c_size_t),     # number of rows
        ("ncol", ctypes.c_size_t),     # number of columns
        ("nzmax", ctypes.c_size_t),    # maximum number of entries
        ("p", ctypes.c_void_p),        # column pointers
        ("i", ctypes.c_void_p),        # row indices
        ("nz", ctypes.c_void_p),        # numeric values
        ("x", ctypes.c_void_p),        # numeric values
        ("z", ctypes.c_void_p),        # complex values
        ("stype", ctypes.c_int),       # symmetric flag
        ("itype", ctypes.c_int),       # index type
        ("xtype", ctypes.c_int),       # real/pattern/complex
        ("dtype", ctypes.c_int),       # data type
        ("sorted", ctypes.c_int),      # columns sorted
        ("packed", ctypes.c_int)       # packed/unpacked
    ]

class cholmod_dense(ctypes.Structure):
    _fields_ = [
        ("nrow", ctypes.c_size_t),     # number of rows
        ("ncol", ctypes.c_size_t),     # number of columns
        ("nzmax", ctypes.c_size_t),    # maximum entries
        ("d", ctypes.c_size_t),        # leading dimension
        ("x", ctypes.c_void_p),        # values
        ("z", ctypes.c_void_p),        # complex values
        ("xtype", ctypes.c_int),       # real/pattern/complex
        ("dtype", ctypes.c_int)        # data type
    ]


    
''' 
FUNCTIONS
'''

libcholmod.cholmod_start.argtypes = [ctypes.POINTER(cholmod_common)]
libcholmod.cholmod_finish.argtypes = [ctypes.POINTER(cholmod_common)]

libcholmod.cholmod_allocate_sparse.argtypes = [
    ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t,
    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int,
    ctypes.POINTER(cholmod_common)
]



libcholmod.cholmod_analyze.argtypes = [
    ctypes.POINTER(cholmod_sparse),
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_factorize.argtypes = [
    ctypes.POINTER(cholmod_sparse),
    ctypes.POINTER(cholmod_factor),
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_solve.argtypes = [
    ctypes.c_int,
    ctypes.POINTER(cholmod_factor),
    ctypes.POINTER(cholmod_dense),
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_allocate_dense.argtypes = [
    ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t, ctypes.c_int,
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_free_sparse.argtypes = [
    ctypes.POINTER(ctypes.POINTER(cholmod_sparse)),
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_free_dense.argtypes = [
    ctypes.POINTER(ctypes.POINTER(cholmod_dense)),
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_free_factor.argtypes = [
    ctypes.POINTER(ctypes.POINTER(cholmod_factor)),
    ctypes.POINTER(cholmod_common)
]

libcholmod.cholmod_norm_sparse.argtypes = [
    ctypes.POINTER(cholmod_sparse),  # A
    ctypes.c_int,                    # norm type: 0=inf, 1=1
    ctypes.POINTER(cholmod_common)   # Common
]

libcholmod.cholmod_start.restype = None
libcholmod.cholmod_finish.restype = None
libcholmod.cholmod_allocate_sparse.restype = ctypes.POINTER(cholmod_sparse)
libcholmod.cholmod_analyze.restype = ctypes.POINTER(cholmod_factor)
libcholmod.cholmod_factorize.restype = ctypes.c_int
libcholmod.cholmod_solve.restype = ctypes.POINTER(cholmod_dense)
libcholmod.cholmod_allocate_dense.restype = ctypes.POINTER(cholmod_dense)
libcholmod.cholmod_free_sparse.restype = None
libcholmod.cholmod_free_dense.restype = None
libcholmod.cholmod_free_factor.restype = None
libcholmod.cholmod_norm_sparse.restype = ctypes.c_double


Helpers

In [ ]:

common = cholmod_common()
libcholmod.cholmod_start(ctypes.byref(common))

csc_matrix.c


def to_cholmod_sparse(A, itype=0, dtype=0):
    """
    Convert a 2D NumPy array A into a cholmod_sparse struct.
    
    Parameters:
        A : np.ndarray
            Dense or 2D array to convert.
        itype : int
            0=int32 indices, 1=int64 indices
        dtype : int
            0=double (real), 1=float (single), 2=complex
    
    Returns:
        cholmod_sparse instance.
    """
    A = np.array(A, copy=False)
    nrow, ncol = A.shape
    
    # Convert to CSC format
    A = csc_matrix(A)
    
    # Prepare contiguous arrays
    x = np.ascontiguousarray(A.data, dtype=np.float64)
    i = np.ascontiguousarray(A.indices, dtype=np.int32 if itype==0 else np.int64)
    p = np.ascontiguousarray(A.indptr, dtype=np.int32 if itype==0 else np.int64)
    
    # Cast to void pointers for ctypes
    x_ptr = x.ctypes.data_as(ctypes.c_void_p)
    i_ptr = i.ctypes.data_as(ctypes.c_void_p)
    p_ptr = p.ctypes.data_as(ctypes.c_void_p)
    
    # Initialize struct
    cholA = cholmod_sparse()
    cholA.nrow = nrow
    cholA.ncol = ncol
    cholA.nzmax = len(x)
    cholA.p = p_ptr
    cholA.i = i_ptr
    cholA.x = x_ptr
    cholA.z = None             # None if real
    cholA.stype = 0            # 0 = general
    cholA.itype = itype
    cholA.xtype = 1            # 1 = real
    cholA.dtype = dtype
    cholA.sorted = 1           # columns sorted
    cholA.packed = 1           # packed
    
    return cholA

def make_dense_vector(values):
    n = len(values)
    D = libcholmod.cholmod_allocate_dense(
        ctypes.c_size_t(n), ctypes.c_size_t(1), ctypes.c_size_t(n),
        ctypes.c_int(1),  # xtype: real
        ctypes.byref(common)
    )
    if not D:
        raise MemoryError("CHOLMOD allocate dense failed")
    arr = (ctypes.c_double * n)(*values)
    D.contents.x = ctypes.cast(arr, ctypes.c_void_p)
    return D


def print_sparse_matrix(A, name, Common):
    # Ensure the name is bytes (C string)
    name_bytes = name.encode('utf-8')
    ret = libcholmod.cholmod_print_sparse(ctypes.byref(A), name_bytes, ctypes.byref(Common)) 
    return ret

def cholmod_norm(A, norm_type, Common):
    """
    Compute a norm of a sparse matrix A.

    Parameters:
        A: cholmod_sparse
        norm_type: int, 0 for infinity norm, 1 for 1-norm
        Common: cholmod_common

    Returns:
        float: the requested norm
    """
    return libcholmod.cholmod_norm_sparse(ctypes.byref(A), norm_type, ctypes.byref(Common))

A_dense = np.array([
    [1, 0, 2],
    [0, 3, 0],
    [4, 0, 5]
])

A_ptr = ctypes.POINTER(cholmod_sparse)()  # initially NULL
A = to_cholmod_sparse(A_dense)
         

# Right-hand side
b = make_dense_vector([1.0,1.0,1.0])


In [ ]:
print("CHOLMOD status:", common.status)
# cholmod_norm(A, 1, common)

# 0 ok
# -4 Invalid input
# -2 out of memory

#define CHOLMOD_A    0  /* solve Ax=b    */
#define CHOLMOD_LDLt 1  /* solve LDL'x=b */
#define CHOLMOD_LD   2  /* solve LDx=b   */
#define CHOLMOD_DLt  3  /* solve DL'x=b  */
#define CHOLMOD_L    4  /* solve Lx=b    */
#define CHOLMOD_Lt   5  /* solve L'x=b   */
#define CHOLMOD_D    6  /* solve Dx=b    */
#define CHOLMOD_P    7  /* permute x=Px  */
#define CHOLMOD_Pt   8  /* permute x=P'x */

In [ ]:
# Constants for cholmod_solve
CHOLMOD_A = 0 
np.set_printoptions(precision=3, suppress=True)
try:

    # Analyze
    factor = libcholmod.cholmod_analyze(ctypes.byref(A), ctypes.byref(common))
    if not factor:
        raise RuntimeError("CHOLMOD analyze failed")
    

    # Factorize
    if libcholmod.cholmod_factorize(ctypes.byref(A), factor, ctypes.byref(common)) == 0:
        raise RuntimeError("CHOLMOD factorize failed")
    

    # Solve
    x = libcholmod.cholmod_solve(CHOLMOD_A, factor, b, ctypes.byref(common))

    # Copy result to numpy
    nsol = x.contents.nrow
    sol_arr = np.ctypeslib.as_array(ctypes.cast(x.contents.x, ctypes.POINTER(ctypes.c_double)), shape=(nsol,))
    print("Solution x:")
    for val in sol_arr:
        print(f"{val:.3f}")

finally:
    libcholmod.cholmod_free_factor(ctypes.byref(factor), ctypes.byref(common))

    # TODO FREE MATRIX AND DENSE
    #libcholmod.cholmod_free_sparse(ctypes.byref(A), ctypes.byref(common))
    #libcholmod.cholmod_free_dense(ctypes.byref(b), ctypes.byref(common))
    libcholmod.cholmod_finish(ctypes.byref(common))